# Imports

In [1]:
import sys
import os

from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession

In [2]:
app_name="Case Study 1"

conf = SparkConf().setAppName(app_name)
sc = SparkContext(conf = conf)
spark = SparkSession(sc)

In [3]:
sc.applicationId

'local-1573654541439'

In [4]:
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)
LOGGER.info("pyspark script logger initialized")

# Load data into Spark DataFrame

In [5]:
def get_hdfs_filepath(file_name, on_cloud=True):
    # path to folder containing this code
    
    prefix = '/data/spark/6_cs1_dataset/'
    if on_cloud:
        bucket  = os.environ['BUCKET']
        file_path = bucket + prefix + file_name
    else:
        file_path = '/Users/val' + prefix + file_name

    return file_path

In [6]:
AISLES = get_hdfs_filepath('aisles.csv')
DEPARTMENTS = get_hdfs_filepath('departments.csv')
ORDER_PRODUCTS_PRIOR = get_hdfs_filepath('order_products__prior.csv')
ORDER_PRODUCTS_TRAIN = get_hdfs_filepath('order_products__train.csv')
ORDERS = get_hdfs_filepath('orders.csv')
PRODUCTS = get_hdfs_filepath('products.csv')

In [7]:
!file '/Users/val/work/data/spark/6_cs1_dataset/aisles.csv'

/Users/val/work/data/spark/6_cs1_dataset/aisles.csv: cannot open `/Users/val/work/data/spark/6_cs1_dataset/aisles.csv' (No such file or directory)


In [8]:
df_aisles = spark.read.csv(AISLES, multiLine=True, header="true",encoding='ASCII', escape= "\"",inferSchema =True)
df_departments = spark.read.csv(DEPARTMENTS, multiLine=True, header="true",encoding='ASCII', escape= "\"",inferSchema =True)
df_order_products_prior = spark.read.csv(ORDER_PRODUCTS_PRIOR, multiLine=True, header="true",encoding='ASCII', escape= "\"",inferSchema =True)
df_order_products_train = spark.read.csv(ORDER_PRODUCTS_TRAIN, multiLine=True, header="true",encoding='ASCII', escape= "\"",inferSchema =True)
df_orders = spark.read.csv(ORDERS, multiLine=True, header="true",encoding='ASCII', escape= "\"",inferSchema =True)
df_products = spark.read.csv(PRODUCTS, multiLine=True, header="true",encoding='ASCII', escape= "\"",inferSchema =True)


# Merge all the data frames based on the common key and create a single DataFrame

### Using dataframe joins

In [9]:
# 1: orders + order_products
orders_train_df = df_orders.join(df_order_products_train, "order_id")
orders_prior_df = df_orders.join(df_order_products_prior, "order_id")

# 2: products + aisles + departments
prod_isles_dep_df = df_departments.join(df_products, "department_id").join(df_aisles, "aisle_id")

# 3: orders + order_products + products + aisles + departments
orders_all_train = prod_isles_dep_df.join(orders_train_df, "product_id")
orders_all_prior = prod_isles_dep_df.join(orders_prior_df, "product_id")

# 4: train + prior
orders_all_df = orders_all_train.union(orders_all_prior)

In [10]:
import time
start_time = time.time()

orders_all_df.limit(20).toPandas()

#print("--- %s seconds ---" % (time.time() - start_time))

,product_id,aisle_id,department_id,department,product_name,aisle,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
0,148,24,4,produce,Nectarines,fresh fruits,704303,90705,train,22,1,14,25.0,9,1
1,148,24,4,produce,Nectarines,fresh fruits,732117,125220,train,6,1,16,21.0,24,0
2,148,24,4,produce,Nectarines,fresh fruits,36580,112691,train,41,0,14,19.0,24,1
3,148,24,4,produce,Nectarines,fresh fruits,1669930,31331,train,14,5,21,21.0,3,1
4,148,24,4,produce,Nectarines,fresh fruits,1524400,11241,train,4,2,11,30.0,10,0
5,148,24,4,produce,Nectarines,fresh fruits,292177,201763,train,37,0,13,8.0,2,1
6,148,24,4,produce,Nectarines,fresh fruits,842159,74316,train,4,0,19,7.0,15,1
7,148,24,4,produce,Nectarines,fresh fruits,3363512,80075,train,42,1,15,4.0,5,1
8,148,24,4,produce,Nectarines,fresh fruits,129966,170386,train,4,6,16,8.0,11,1
9,148,24,4,produce,Nectarines,fresh fruits,1337576,37289,train,20,3,13,12.0,5,1


### Using spark.sql querries

In [11]:
df_aisles.createOrReplaceTempView('Aisles')
df_departments.createOrReplaceTempView('Departments')
df_order_products_prior.createOrReplaceTempView('Order_products_prior')
df_order_products_train.createOrReplaceTempView('Order_products_train')
df_orders.createOrReplaceTempView('Orders')
df_products.createOrReplaceTempView('Products')


In [12]:
querry = """
Select
  m1.order_id,
  m1.user_id,
  m1.eval_set,
  m1.order_number,
  m1.order_dow,
  m1.order_hour_of_day,
  m1.days_since_prior_order,
  m1.product_id,
  m1.add_to_cart_order,
  m1.reordered,
  m2.product_name,
  m2.aisle_id,
  m2.aisle,
  m2.department_id,
  m2.department 
From
  (
    Select
      o.order_id,
      o.user_id,
      o.eval_set,
      o.order_number,
      o.order_dow,
      o.order_hour_of_day,
      o.days_since_prior_order,
      op.product_id,
      op.add_to_cart_order,
      op.reordered 
    From
      Orders o 
      INNER JOIN
        (
          SELECT
            * 
          FROM
            Order_products_train 
          UNION ALL
          SELECT
            * 
          FROM
            Order_products_prior
        )
        op 
        on o.order_id = op.order_id 
  )
  m1 
  INNER JOIN
    (
      Select
        p.product_id,
        p.product_name,
        p.aisle_id,
        a.aisle,
        p.department_id,
        d.department 
      From
        Departments d 
        INNER JOIN
          Products p 
          on d.department_id = p.department_id 
        INNER JOIN
          Aisles a 
          on p.aisle_id = a.aisle_id 
    )
    m2 
    on m1.product_id = m2.product_id
"""

In [13]:
orders_all_df = spark.sql(querry)
orders_all_df.createOrReplaceTempView('Orders_all')

In [14]:
import time
start_time = time.time()

orders_all_df.limit(20).toPandas()

#print("--- %s seconds ---" % (time.time() - start_time))


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,aisle,department_id,department
0,133160,47136,prior,3,0,18,30.0,148,26,0,Nectarines,24,fresh fruits,4,produce
1,210175,12428,prior,14,5,15,7.0,148,15,1,Nectarines,24,fresh fruits,4,produce
2,295286,63587,prior,1,3,22,NaN,148,10,0,Nectarines,24,fresh fruits,4,produce
3,407324,158452,prior,11,2,9,13.0,148,11,0,Nectarines,24,fresh fruits,4,produce
4,544414,82360,prior,3,6,20,6.0,148,17,0,Nectarines,24,fresh fruits,4,produce
5,602118,157841,prior,12,4,14,9.0,148,3,1,Nectarines,24,fresh fruits,4,produce
6,716590,8196,prior,1,4,17,NaN,148,27,0,Nectarines,24,fresh fruits,4,produce
7,730383,30910,prior,3,5,21,7.0,148,31,1,Nectarines,24,fresh fruits,4,produce
8,734099,122073,prior,9,6,7,20.0,148,6,1,Nectarines,24,fresh fruits,4,produce
9,1074455,145986,prior,6,3,16,8.0,148,11,1,Nectarines,24,fresh fruits,4,produce


# Check missing data

In [15]:
orders_all_df

DataFrame[order_id: int, user_id: int, eval_set: string, order_number: int, order_dow: int, order_hour_of_day: int, days_since_prior_order: double, product_id: int, add_to_cart_order: int, reordered: int, product_name: string, aisle_id: int, aisle: string, department_id: int, department: string]

In [16]:
# ckeck if null and NaN values are present in our DF
from pyspark.sql.functions import isnan, when, count, col

orders_all_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in orders_all_df.columns]).toPandas()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,aisle,department_id,department
0,0,0,0,0,0,0,2078068,0,0,0,0,0,0,0,0


In [17]:
orders_all_df.count()

33819106

In [18]:
orders_all_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- eval_set: string (nullable = true)
 |-- order_number: integer (nullable = true)
 |-- order_dow: integer (nullable = true)
 |-- order_hour_of_day: integer (nullable = true)
 |-- days_since_prior_order: double (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: integer (nullable = true)
 |-- aisle: string (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- department: string (nullable = true)



# List the most ordered products (top 10)

### Using spark.sql querries

In [19]:
querry1 = """
Select
  product_id,
  product_name,
  SUM(add_to_cart_order)
FROM
  Orders_all 
GROUP BY
  product_id,
  product_name 
ORDER BY
  SUM(add_to_cart_order) DESC
"""
most_ordered_df = spark.sql(querry1)
#most_ordered_df.createOrReplaceTempView('Most_ordered')

In [20]:
start_time = time.time()

most_ordered_df.show(10)

print("--- %s seconds ---" % (time.time() - start_time))

+----------+--------------------+----------------------+
|product_id|        product_name|sum(add_to_cart_order)|
+----------+--------------------+----------------------+
|     24852|              Banana|               2405664|
|     13176|Bag of Organic Ba...|               2007396|
|     21137|Organic Strawberries|               2005999|
|     21903|Organic Baby Spinach|               1870623|
|     47209|Organic Hass Avocado|               1497887|
|     47626|         Large Lemon|               1279007|
|     26209|               Limes|               1261016|
|     47766|     Organic Avocado|               1187054|
|     16797|        Strawberries|               1065577|
|     27966| Organic Raspberries|               1031044|
+----------+--------------------+----------------------+
only showing top 10 rows

--- 32.8523211479187 seconds ---


### Using dataframe joins

In [21]:
from pyspark.sql.functions import sum
start_time = time.time()

orders_all_df.select("product_id", "product_name","add_to_cart_order").groupBy("product_id", "product_name").agg(sum("add_to_cart_order")).orderBy(sum("add_to_cart_order"),ascending=False).show(10)

print("--- %s seconds ---" % (time.time() - start_time))


+----------+--------------------+----------------------+
|product_id|        product_name|sum(add_to_cart_order)|
+----------+--------------------+----------------------+
|     24852|              Banana|               2405664|
|     13176|Bag of Organic Ba...|               2007396|
|     21137|Organic Strawberries|               2005999|
|     21903|Organic Baby Spinach|               1870623|
|     47209|Organic Hass Avocado|               1497887|
|     47626|         Large Lemon|               1279007|
|     26209|               Limes|               1261016|
|     47766|     Organic Avocado|               1187054|
|     16797|        Strawberries|               1065577|
|     27966| Organic Raspberries|               1031044|
+----------+--------------------+----------------------+
only showing top 10 rows

--- 32.087281942367554 seconds ---


# Do people usually reorder the same previously ordered products?

In [22]:
querry5 ="""
Select t1.user_id, t1.count as no_of_reordered_product_ids, t2.count as no_of_new_product_ids, t1.count/(t1.count+t2.count) as percent_reordered
FROM 
    (
    Select 
      user_id,
      COUNT(product_id) as count
    FROM
      Orders_all
    WHERE
      reordered = 1
    GROUP BY
      user_id
    ) t1
    JOIN
    (
    Select 
      user_id,
      COUNT(product_id) as count
    FROM
      Orders_all
    WHERE
      reordered = 0
    GROUP BY
      user_id
    ) t2
    ON t1.user_id=t2.user_id
ORDER BY 
 percent_reordered DESC
"""
reordered5_df = spark.sql(querry5)

In [23]:
reordered5_df.limit(40).show(40)

+-------+---------------------------+---------------------+------------------+
|user_id|no_of_reordered_product_ids|no_of_new_product_ids| percent_reordered|
+-------+---------------------------+---------------------+------------------+
|  99753|                        191|                    2|0.9896373056994818|
|  82414|                        425|                    8|0.9815242494226328|
| 107528|                        104|                    2|0.9811320754716981|
|  17997|                        426|                    9|0.9793103448275862|
| 170174|                         47|                    1|0.9791666666666666|
|   3269|                        272|                    6|0.9784172661870504|
|   5588|                       2178|                   50|0.9775583482944344|
|  91160|                        207|                    5|0.9764150943396226|
| 184517|                        205|                    5|0.9761904761904762|
|  26489|                        186|               

In [24]:
from pyspark.sql.functions import *
result = reordered5_df.agg(avg('no_of_reordered_product_ids'), avg('no_of_new_product_ids'))
result.show()

+--------------------------------+--------------------------+
|avg(no_of_reordered_product_ids)|avg(no_of_new_product_ids)|
+--------------------------------+--------------------------+
|                97.6724471266562|         67.70212276502748|
+--------------------------------+--------------------------+



Since avg(no_of_reordered_product_ids) is greater than avg(no_of_new_product_ids), we conclude that the answer is YES: people are 30% more likely to reorder same items than to buy something new.

### Alternative approach

In [25]:
querry3 = """
Select
  t1.product_id,
  t1.product_name,
  t1.s1 / (t2.s2 + t1.s1) AS percent_units_reordered,
  t1.s1 + t2.s2 as units_sold,
  CASE
    WHEN
      t1.s1 / (t2.s2 + t1.s1) >= 0.75 
    THEN
      "YES" 
    WHEN
      t1.s1 / (t2.s2 + t1.s1) < 0.75 
      AND t1.s1 / (t2.s2 + t1.s1) >= 0.5 
    THEN
      "MAYBE" 
    WHEN
      t1.s1 / (t2.s2 + t1.s1) < 0.5 
      AND t1.s1 / (t2.s2 + t1.s1) >= 0.25 
    THEN
      "NOT REALLY" 
    ELSE
      "NO" 
  END
  AS ReorderedText 
FROM
  (
    Select
      product_id,
      product_name,
      SUM(add_to_cart_order) as s1 
    FROM
      Orders_all 
    WHERE
      reordered = 1 
    GROUP BY
      product_id,
      product_name 
  )
  t1 
  FULL OUTER JOIN
    (
      Select
        product_id,
        product_name,
        SUM(add_to_cart_order) as s2 
      FROM
        Orders_all 
      WHERE
        reordered = 0 
      GROUP BY
        product_id,
        product_name 
    )
    t2 
    ON t1.product_id = t2.product_id 
ORDER BY
  percent_units_reordered DESC
"""

reordered1_df = spark.sql(querry3)

In [26]:
from pyspark.sql.window import Window
df = reordered1_df
window = Window.partitionBy(
    df['ReorderedText']
).orderBy(
    df['percent_units_reordered'].desc()
)

df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 10).orderBy(
    df['percent_units_reordered'].desc()
).limit(40).toPandas()

,product_id,product_name,percent_units_reordered,units_sold,ReorderedText,rank
0,9889,Dietary Supplement Tablets for Normal Sleep Pa...,0.956522,46,YES,1
1,43553,Orange Energy Shots,0.956522,46,YES,1
2,39992,"Energy Shot, Grape Flavor",0.949153,118,YES,3
3,14609,Soy Powder Infant Formula,0.943396,53,YES,4
4,802,Organic Super Berry Juice,0.936620,426,YES,5
5,35910,Porter,0.932039,206,YES,6
6,11082,Simtra Triple IPA,0.931751,674,YES,7
7,44015,Classic Citrus Kombucha,0.926829,123,YES,8
8,44951,California Ripe Pitted Extra Large Olives,0.923077,39,YES,9
9,22096,California French Dressing,0.922222,90,YES,10


In [27]:
from pyspark.sql.window import Window
df = reordered1_df
window = Window.partitionBy(
    df['ReorderedText']
).orderBy(
    df['units_sold'].desc()
)

df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 10).orderBy(
    df['units_sold'].desc()
).limit(40).toPandas()

,product_id,product_name,percent_units_reordered,units_sold,ReorderedText,rank
0,24852,Banana,0.814670,2405664,YES,1
1,13176,Bag of Organic Bananas,0.798724,2007396,YES,2
2,21137,Organic Strawberries,0.764337,2005999,YES,3
3,21903,Organic Baby Spinach,0.757061,1870623,YES,4
4,47209,Organic Hass Avocado,0.773151,1497887,YES,5
5,47626,Large Lemon,0.669930,1279007,MAYBE,1
6,26209,Limes,0.659409,1261016,MAYBE,2
7,47766,Organic Avocado,0.729510,1187054,MAYBE,3
8,16797,Strawberries,0.660797,1065577,MAYBE,4
9,27966,Organic Raspberries,0.723778,1031044,MAYBE,5


# List most reordered products

This approach takes into consideration number of items added to cart_order as well as number of orders.

In [28]:
reordered1_df.limit(30).show(30)

+----------+--------------------+-----------------------+----------+-------------+
|product_id|        product_name|percent_units_reordered|units_sold|ReorderedText|
+----------+--------------------+-----------------------+----------+-------------+
|      9889|Dietary Supplemen...|     0.9565217391304348|        46|          YES|
|     43553| Orange Energy Shots|     0.9565217391304348|        46|          YES|
|     39992|Energy Shot, Grap...|     0.9491525423728814|       118|          YES|
|     14609|Soy Powder Infant...|     0.9433962264150944|        53|          YES|
|       802|Organic Super Ber...|     0.9366197183098591|       426|          YES|
|     35910|              Porter|     0.9320388349514563|       206|          YES|
|     11082|   Simtra Triple IPA|     0.9317507418397626|       674|          YES|
|     44015|Classic Citrus Ko...|      0.926829268292683|       123|          YES|
|     44951|California Ripe P...|     0.9230769230769231|        39|          YES|
|   

### Alternative approach

This approach based solely on number of orders and does not consider number of items in cart_order.

In [29]:
from pyspark.sql.functions import *
orders_all_df.groupBy("product_name").agg(
    avg("reordered").alias("percent_reorders"), 
    count("reordered").alias('no_of_orders'),
    sum("add_to_cart_order").alias("untis_sold"),  
    (sum("add_to_cart_order")/countDistinct("order_id")).alias('units_per_order')
).sort(
    col("percent_reorders").desc()
).limit(20).toPandas()



,product_name,percent_reorders,no_of_orders,untis_sold,units_per_order
0,Raw Veggie Wrappers,0.942029,69,185,2.681159
1,Serenity Ultimate Extrema Overnight Pads,0.933333,90,429,4.766667
2,Orange Energy Shots,0.923077,13,46,3.538462
3,Chocolate Love Bar,0.921569,102,375,3.676471
4,Soy Powder Infant Formula,0.914286,35,53,1.514286
5,Simply Sleep Nighttime Sleep Aid,0.911111,45,144,3.200000
6,"Energy Shot, Grape Flavor",0.909091,22,118,5.363636
7,Russian River Valley Reserve Pinot Noir,0.900000,30,70,2.333333
8,Bars Peanut Butter,0.898551,69,426,6.173913
9,Soy Crisps Lightly Salted,0.895522,67,246,3.671642


# Most important department and aisle (by number of products)

In [30]:
# 2: products + aisles + departments
prod_isles_dep_df = df_departments.join(df_products, "department_id").join(df_aisles, "aisle_id")



In [31]:
prod_isles_dep_df.groupBy("department").count().sort(col("count").desc()).show(10,False)
prod_isles_dep_df.groupBy("aisle").count().sort(col("count").desc()).show(10,False)

+---------------+-----+
|department     |count|
+---------------+-----+
|personal care  |6563 |
|snacks         |6264 |
|pantry         |5371 |
|beverages      |4365 |
|frozen         |4007 |
|dairy eggs     |3449 |
|household      |3085 |
|canned goods   |2092 |
|dry goods pasta|1858 |
|produce        |1684 |
+---------------+-----+
only showing top 10 rows

+--------------------+-----+
|aisle               |count|
+--------------------+-----+
|missing             |1258 |
|candy chocolate     |1246 |
|ice cream ice       |1091 |
|vitamins supplements|1038 |
|yogurt              |1026 |
|chips pretzels      |989  |
|tea                 |894  |
|packaged cheese     |891  |
|frozen meals        |880  |
|cookies cakes       |874  |
+--------------------+-----+
only showing top 10 rows



# Get the Top 10 departments

In [32]:
orders_all_df.groupBy("department").agg(sum("reordered").alias("no_of_orders")).sort(col("no_of_orders").desc()).show(10)

+------------+------------+
|  department|no_of_orders|
+------------+------------+
|     produce|     6432596|
|  dairy eggs|     3773723|
|   beverages|     1832952|
|      snacks|     1727075|
|      frozen|     1268058|
|      bakery|      769880|
|      pantry|      679799|
|        deli|      666231|
|canned goods|      511317|
|meat seafood|      420349|
+------------+------------+
only showing top 10 rows



# List top 10 products ordered in the morning (6 AM to 11 AM)

In [33]:
orders_all_df.where(
    (orders_all_df["order_hour_of_day"]>6) & (orders_all_df["order_hour_of_day"]<11)
).groupBy("product_name").agg(
    sum("reordered").alias("no_of_orders"),sum("add_to_cart_order").alias("no_of_units")
).sort(col("no_of_units").desc()).show(10)



+--------------------+------------+-----------+
|        product_name|no_of_orders|no_of_units|
+--------------------+------------+-----------+
|              Banana|      107870|     605101|
|Organic Strawberries|       53909|     503409|
|Bag of Organic Ba...|       84433|     501032|
|Organic Baby Spinach|       47174|     457051|
|Organic Hass Avocado|       42877|     367110|
|         Large Lemon|       27373|     313270|
|               Limes|       23580|     301298|
|     Organic Avocado|       33308|     281429|
|        Strawberries|       27875|     274751|
| Organic Raspberries|       29273|     268852|
+--------------------+------------+-----------+
only showing top 10 rows

